# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacja|recommendation|span.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|span.user-post__score-count|
|treść opinii|content|div.user-post__text|
|listę zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|listę wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|opinion_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|button.vote-no > span|

In [22]:
selectors = {
    'author': ("span.user-post__author-name",),
    'recommendation': ("span.user-post__author-recomendation > em",),
    'rating': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    'opinion_date': ("span.user-post__published > time:nth-child(1)","datetime"),
    'purchase_date': ("span.user-post__published > time:nth-child(2)","datetime"),
    'likes': ("button.vote-yes > span",),
    'dislikes': ("button.vote-no > span",),
    'opinion_id': (None, "data-entry-id"),
}

1. Import bibliotek

In [23]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. Funkcja do ekstrakcji zawartości ze strony HTML

In [24]:
def extract(ancestor, selector, attribute = None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return  ancestor[attribute].strip()
    return ancestor.get_text().strip()

2. Adres pierwszej Strony z opiniomi o produkcie

In [25]:
# product_id = "84514582"
product_id = input("Podaj kod produktu z Ceneo.pl: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

5. Pobranie opinii o producie

In [26]:
all_opinions = []
while(url): 
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", "href")
    except TypeError:
        url = None

https://www.ceneo.pl/157145069#tab=reviews
https://www.ceneo.pl/157145069/opinie-2
https://www.ceneo.pl/157145069/opinie-3
https://www.ceneo.pl/157145069/opinie-4
https://www.ceneo.pl/157145069/opinie-5
https://www.ceneo.pl/157145069/opinie-6
https://www.ceneo.pl/157145069/opinie-7
https://www.ceneo.pl/157145069/opinie-8
https://www.ceneo.pl/157145069/opinie-9
https://www.ceneo.pl/157145069/opinie-10
https://www.ceneo.pl/157145069/opinie-11
https://www.ceneo.pl/157145069/opinie-12
https://www.ceneo.pl/157145069/opinie-13
https://www.ceneo.pl/157145069/opinie-14
https://www.ceneo.pl/157145069/opinie-15
https://www.ceneo.pl/157145069/opinie-16
https://www.ceneo.pl/157145069/opinie-17
https://www.ceneo.pl/157145069/opinie-18
https://www.ceneo.pl/157145069/opinie-19
https://www.ceneo.pl/157145069/opinie-20
https://www.ceneo.pl/157145069/opinie-21
https://www.ceneo.pl/157145069/opinie-22
https://www.ceneo.pl/157145069/opinie-23
https://www.ceneo.pl/157145069/opinie-24
https://www.ceneo.pl/1

In [27]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)